# --- Day 3: Gear Ratios ---

## Part 1

You and the Elf eventually reach a gondola lift station; he says the gondola lift will take you up to the water source, but this is as far as he can bring you. You go inside.

It doesn't take long to find the gondolas, but there seems to be a problem: they're not moving.

"Aaah!"

You turn around to see a slightly-greasy Elf with a wrench and a look of surprise. "Sorry, I wasn't expecting anyone! The gondola lift isn't working right now; it'll still be a while before I can fix it." You offer to help.

The engineer explains that an engine part seems to be missing from the engine, but nobody can figure out which one. If you can add up all the part numbers in the engine schematic, it should be easy to work out which part is missing.

The engine schematic (your puzzle input) consists of a visual representation of the engine. There are lots of numbers and symbols you don't really understand, but apparently any number adjacent to a symbol, even diagonally, is a "part number" and should be included in your sum. (Periods (.) do not count as a symbol.)

Here is an example engine schematic:

```
467..114..
...*......
..35..633.
......#...
617*......
.....+.58.
..592.....
......755.
...$.*....
.664.598..
```

In this schematic, two numbers are not part numbers because they are not adjacent to a symbol: 114 (top right) and 58 (middle right). Every other number is adjacent to a symbol and so is a part number; their sum is 4361.

In [179]:
from math import prod

input = "467..114..\n\
...*......\n\
..35..633.\n\
....../...\n\
617*......\n\
.....@.58.\n\
..592.....\n\
......755.\n\
...$.*....\n\
.664.598..\n\
".splitlines()

input = open("input.txt").read().splitlines()

def contains_non_digit_non_dot(lst):
    return any(not (s.isdigit() or s == '.') for s in lst)

def get_chars_to_check(x,y,i,matrix):
    chars = []
    left_x = x-len(i)+1
    # Middle
    if left_x > 0 and x < len(matrix[0])-1 and y > 0 and y < len(matrix)-1:
        # 4 corners
        chars.append(matrix[y-1][left_x-1])
        chars.append(matrix[y-1][x+1])
        chars.append(matrix[y+1][left_x-1])
        chars.append(matrix[y+1][x+1])
        # 2 laterals
        chars.append(matrix[y][left_x-1])
        chars.append(matrix[y][x+1])
        # Up & Down
        chars += [matrix[y-1][xx] for xx in [j for j in range(left_x,x+1)]]
        chars += [matrix[y+1][xx] for xx in [j for j in range(left_x,x+1)]]
    # First column without corners
    elif left_x == 0 and y > 0 and y < len(matrix)-1:
        # 2 right corners
        chars.append(matrix[y-1][x+1])
        chars.append(matrix[y+1][x+1])
        # righ lateral
        chars.append(matrix[y][x+1])
        # Up & Down
        chars += [matrix[y-1][xx] for xx in [j for j in range(left_x,x+1)]]
        chars += [matrix[y+1][xx] for xx in [j for j in range(left_x,x+1)]]
    # Last column without corners
    elif x == len(matrix[0])-1 and y > 0 and y < len(matrix)-1:
        # 2 left corners
        chars.append(matrix[y-1][left_x-1])
        chars.append(matrix[y+1][left_x-1])
        # left lateral
        chars.append(matrix[y][left_x-1])
        # Up & Down
        chars += [matrix[y-1][xx] for xx in [j for j in range(left_x,x+1)]]
        chars += [matrix[y+1][xx] for xx in [j for j in range(left_x,x+1)]]
    # First row without corners
    elif left_x > 0 and x < len(matrix[0])-1 and y == 0:
        # 2 lower corners
        chars.append(matrix[y+1][left_x-1])
        chars.append(matrix[y+1][x+1])
        # 2 laterals
        chars.append(matrix[y][left_x-1])
        chars.append(matrix[y][x+1])
        # Down
        chars += [matrix[y+1][xx] for xx in [j for j in range(left_x,x+1)]]
    # Last row without corners
    elif left_x > 0 and x < len(matrix[0])-1 and y == len(matrix)-1:
        # 2 upper corners
        chars.append(matrix[y-1][left_x-1])
        chars.append(matrix[y-1][x+1])
        # 2 laterals
        chars.append(matrix[y][left_x-1])
        chars.append(matrix[y][x+1])
        # Up
        chars += [matrix[y-1][xx] for xx in [j for j in range(left_x,x+1)]]
    # We assume no number fits whole line
    # Upper-Left Corner
    elif left_x == 0 and y == 0:
        # lower-right corner
        chars.append(matrix[y+1][x+1])
        # Right lateral
        chars.append(matrix[y][x+1])
        # Down
        chars += [matrix[y+1][xx] for xx in [j for j in range(left_x,x+1)]]
    # Lower-Left Corner
    elif left_x == 0 and y == len(matrix)-1:
        # upper-right corner
        chars.append(matrix[y-1][x+1])
        # Right lateral
        chars.append(matrix[y][x+1])
        # Up
        chars += [matrix[y-1][xx] for xx in [j for j in range(left_x,x+1)]]
    # Upper-Right Corner
    elif x == len(matrix[0])-1 and y == 0:
        # lower-left corner
        chars.append(matrix[y+1][left_x-1])
        # Left lateral
        chars.append(matrix[y][left_x-1])
        # Down
        chars += [matrix[y+1][xx] for xx in [j for j in range(left_x,x+1)]]
    # Lower-Right Corner
    elif x == len(matrix[0])-1 and y == len(matrix)-1:
        # upper-left corner
        chars.append(matrix[y-1][left_x-1])
        # Left lateral
        chars.append(matrix[y][left_x-1])
        # Up
        chars += [matrix[y-1][xx] for xx in [j for j in range(left_x,x+1)]]
    
    return chars

matrix = [list(row) for row in input]

pieces = []

for y in range(len(matrix)):
    i = ''
    #y=49
    for x in range(len(matrix[0])):
        if matrix[y][x].isdigit():
            i += matrix[y][x]
            if x==len(matrix[0])-1:
                chars_to_check = get_chars_to_check(x,y,i,matrix)
                for c in chars_to_check:
                    if contains_non_digit_non_dot(c):
                        pieces.append(int(i))
                i=''
        else:
            if len(i)>0:
                chars_to_check = get_chars_to_check(x-1,y,i,matrix)
                for c in chars_to_check:
                    if contains_non_digit_non_dot(c):
                        pieces.append(int(i))
                i=''

print("Total sum of game IDs:", sum(pieces))




Total sum of game IDs: 527144


## --- Part Two ---

The engineer finds the missing part and installs it in the engine! As the engine springs to life, you jump in the closest gondola, finally ready to ascend to the water source.

You don't seem to be going very fast, though. Maybe something is still wrong? Fortunately, the gondola has a phone labeled "help", so you pick it up and the engineer answers.

Before you can explain the situation, she suggests that you look out the window. There stands the engineer, holding a phone in one hand and waving with the other. You're going so slowly that you haven't even left the station. You exit the gondola.

The missing part wasn't the only issue - one of the gears in the engine is wrong. A gear is any * symbol that is adjacent to exactly two part numbers. Its gear ratio is the result of multiplying those two numbers together.

This time, you need to find the gear ratio of every gear and add them all up so that the engineer can figure out which gear needs to be replaced.

Consider the same engine schematic again:

```
467..114..
...*......
..35..633.
......#...
617*......
.....+.58.
..592.....
......755.
...$.*....
.664.598..
```

In this schematic, there are two gears. The first is in the top left; it has part numbers 467 and 35, so its gear ratio is 16345. The second gear is in the lower right; its gear ratio is 451490. (The * adjacent to 617 is not a gear because it is only adjacent to one part number.) Adding up all of the gear ratios produces 467835.

> Part 2 Not solved, it works with the example only

In [209]:
input = "467..114..\n\
...*......\n\
..35..633.\n\
......#...\n\
617*......\n\
.....+.58.\n\
..592.....\n\
......755.\n\
...$.*....\n\
.664.598..\n\
".splitlines()


input = open("input.txt").read().splitlines()

def contains_non_digit_non_dot(lst):
    return any(not (s.isdigit() or s == '.') for s in lst)

def get_chars_to_check(x,y,i,matrix):
    chars = []
    left_x = x-len(i)+1
    #print(str(left_x)+'-'+str(x),y,i)
    # Middle
    if left_x > 0 and x < len(matrix[0])-1 and y > 0 and y < len(matrix)-1:
        # 4 corners
        # print(f"[{left_x-1}][{y-1}] {matrix[y-1][left_x-1]}")
        # print(f"[{x+1}][{y-1}] {matrix[y-1][x+1]}")
        # print(f"[{left_x-1}][{y+1}] {matrix[y+1][left_x-1]}")
        # print(f"[{x+1}][{y+1}] {matrix[y+1][x+1]}")
        chars.append(matrix[y-1][left_x-1])
        chars.append(matrix[y-1][x+1])
        chars.append(matrix[y+1][left_x-1])
        chars.append(matrix[y+1][x+1])
        # 2 laterals
        chars.append(matrix[y][left_x-1])
        chars.append(matrix[y][x+1])
        # Up & Down
        chars += [matrix[y-1][xx] for xx in [j for j in range(left_x,x+1)]]
        chars += [matrix[y+1][xx] for xx in [j for j in range(left_x,x+1)]]
    # First column without corners
    elif left_x == 0 and y > 0 and y < len(matrix)-1:
        # 2 right corners
        chars.append(matrix[y-1][x+1])
        chars.append(matrix[y+1][x+1])
        # righ lateral
        #print(f"[{x+1}][{y}] {matrix[y][x+1]}")
        chars.append(matrix[y][x+1])
        # Up & Down
        chars += [matrix[y-1][xx] for xx in [j for j in range(left_x,x+1)]]
        chars += [matrix[y+1][xx] for xx in [j for j in range(left_x,x+1)]]
    # Last column without corners
    elif x == len(matrix[0])-1 and y > 0 and y < len(matrix)-1:
        # 2 left corners
        chars.append(matrix[y-1][left_x-1])
        chars.append(matrix[y+1][left_x-1])
        # left lateral
        chars.append(matrix[y][left_x-1])
        # Up & Down
        chars += [matrix[y-1][xx] for xx in [j for j in range(left_x,x+1)]]
        chars += [matrix[y+1][xx] for xx in [j for j in range(left_x,x+1)]]
    # First row without corners
    elif left_x > 0 and x < len(matrix[0])-1 and y == 0:
        # 2 lower corners
        chars.append(matrix[y+1][left_x-1])
        chars.append(matrix[y+1][x+1])
        # 2 laterals
        chars.append(matrix[y][left_x-1])
        chars.append(matrix[y][x+1])
        # Down
        #print([(y+1,xx) for xx in [j for j in range(left_x,x+1)]])
        #print([matrix[y+1][xx] for xx in [j for j in range(left_x,x+1)]])
        chars += [matrix[y+1][xx] for xx in [j for j in range(left_x,x+1)]]
    # Last row without corners
    elif left_x > 0 and x < len(matrix[0])-1 and y == len(matrix)-1:
        # 2 upper corners
        chars.append(matrix[y-1][left_x-1])
        chars.append(matrix[y-1][x+1])
        # 2 laterals
        chars.append(matrix[y][left_x-1])
        chars.append(matrix[y][x+1])
        # Up
        chars += [matrix[y-1][xx] for xx in [j for j in range(left_x,x+1)]]
    # We assume no number fits whole line
    # Upper-Left Corner
    elif left_x == 0 and y == 0:
        # lower-right corner
        chars.append(matrix[y+1][x+1])
        # Right lateral
        chars.append(matrix[y][x+1])
        # Down
        chars += [matrix[y+1][xx] for xx in [j for j in range(left_x,x+1)]]
    # Lower-Left Corner
    elif left_x == 0 and y == len(matrix)-1:
        # upper-right corner
        print(x,y)
        print(matrix)
        chars.append(matrix[y-1][x+1])
        # Right lateral
        chars.append(matrix[y][x+1])
        # Up
        chars += [matrix[y-1][xx] for xx in [j for j in range(left_x,x+1)]]
    # Upper-Right Corner
    elif x == len(matrix[0])-1 and y == 0:
        # lower-left corner
        chars.append(matrix[y+1][left_x-1])
        # Left lateral
        chars.append(matrix[y][left_x-1])
        # Down
        chars += [matrix[y+1][xx] for xx in [j for j in range(left_x,x+1)]]
    # Lower-Right Corner
    elif x == len(matrix[0])-1 and y == len(matrix)-1:
        # upper-left corner
        chars.append(matrix[y-1][left_x-1])
        # Left lateral
        chars.append(matrix[y][left_x-1])
        # Up
        chars += [matrix[y-1][xx] for xx in [j for j in range(left_x,x+1)]]
    
    #print(chars,'\n\n')
    return chars

matrix = [list(row) for row in input]

pieces = []

def check_pieces(matrix):
    for y in range(len(matrix)):
        i = ''
        #y=49
        for x in range(len(matrix[0])):
            if matrix[y][x].isdigit():
                i += matrix[y][x]
                if x==len(matrix[0])-1:
                    chars_to_check = get_chars_to_check(x,y,i,matrix)
                    for c in chars_to_check:
                        if contains_non_digit_non_dot(c):
                            pieces.append(int(i))
                    i=''
            else:
                if len(i)>0:
                    chars_to_check = get_chars_to_check(x-1,y,i,matrix)
                    for c in chars_to_check:
                        if contains_non_digit_non_dot(c):
                            pieces.append(int(i))
                    i=''
    return pieces

def get_i(x,y,matrix):
    c = ''
    for i in range(1,len(matrix[0])):
        if matrix[y-1][x-i].isdigit():
                pass
        else:
            return i-1
def get_j(x,y,matrix):
    c = ''
    for j in range(1,len(matrix[0])):
        #print(matrix[y-1][x+j])
        if matrix[y-1][x+j].isdigit():
                continue
        else:
            return j+1


gears = []
for y in range(len(matrix)):
    for x in range(len(matrix[0])):
        if matrix[y][x]=='*':
            chars_to_check = get_chars_to_check(x,y,'*',matrix)
            if sum([c.isdigit() for c in chars_to_check])>1: #at least 2 numbers
                print(x,y,'*',chars_to_check)
                # if y==0: y+=1
                # if y==len(matrix): y-=1
                # if x==0: x+=1
                # if x==len(matrix[0]): x-=1
                #print(x,y,'*',chars_to_check)
                i = get_i(x,y,matrix)
                j = get_j(x,y,matrix)
                print(i,j)
                print(x-i,x+j)
                if y>0 and len(matrix[0])-1:
                    print("heeey")
                    submatrix = [list(row[x-i:x+j]) for row in matrix[y-1:y+2]]
                else:
                    if y<len(matrix[0])-1:
                        print("heeeeeeeeeey")
                        submatrix = [list(row[x-i:x+j]) for row in matrix[y-1:y+2]]
                    else:
                        print("hoooooy")
                        submatrix = [list(row[x-i:x+j]) for row in matrix[y-1:y+1]]
                print(submatrix)
                #print(submatrix)
                pieces = check_pieces(submatrix)
                if len(pieces)==2:
                    #print(pieces)
                    gears.append(prod(pieces))
                    pieces=[]
                #print("\n")

#print(len(matrix),len(matrix[0]))
#print(pieces)
#print(matrix)

print("Total sum of the gear ratios in the engine schematic:", sum(gears))




7 1 * ['1', '.', '9', '.', '.', '.', '3', '.']
2 2
5 9
heeey
[['6', '1', '3', '.'], ['.', '.', '*', '.'], ['3', '9', '.', '.']]
1 2
[['6', '1', '3', '.'], ['.', '.', '*', '.'], ['3', '9', '.', '.']]
44 1 * ['4', '9', '.', '6', '.', '.', '3', '.']
1 3
43 47
heeey
[['4', '3', '9', '.'], ['.', '*', '.', '.'], ['.', '.', '6', '9']]
89 1 * ['8', '.', '.', '.', '.', '4', '.', '.']
3 2
86 91
heeey
[['4', '3', '8', '.', '.'], ['.', '.', '.', '*', '4'], ['.', '=', '.', '.', '.']]
94 1 * ['.', '6', '.', '4', '.', '.', '.', '.']
0 5
94 99
heeey
[['.', '6', '1', '7', '.'], ['*', '.', '.', '.', '.'], ['.', '4', '0', '4', '.']]
104 1 * ['4', '.', '2', '.', '.', '.', '3', '.']
2 2
102 106
heeey
[['3', '4', '3', '.'], ['.', '.', '*', '.'], ['7', '2', '.', '.']]
1 2
[['3', '4', '3', '.'], ['.', '.', '*', '.'], ['7', '2', '.', '.']]
118 1 * ['.', '4', '3', '.', '.', '.', '9', '7']
0 4
118 122
heeey
[['9', '4', '2', '.'], ['*', '.', '.', '.'], ['7', '.', '.', '.']]
0 2
[['9', '4', '2', '.'], ['*', '.', '

IndexError: list index out of range